# Plotting with pychromatic

pychromatic makes it easy to create publication-quality plots with curated
colour palettes. This page covers `Multiplot` for subplot layouts,
`BrokenAxes` for discontinuous data, and the `chromatify` decorator.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pychromatic import Multiplot, BrokenAxes, Palette, chromatify

## Quick look: Palettes

A `Palette` groups a set of named colours. pychromatic ships 22 built-in
palettes (see the [Palettes page](palettes.ipynb) for the complete list).
Here is a quick preview — the `show()` method renders a colour swatch.

In [ ]:
p = Palette("default")
p.show()

In [ ]:
p = Palette("rainbow")
p.show()

Palettes integrate directly with matplotlib. Individual hex colours are
available via indexing or iteration:

In [ ]:
p = Palette("set5")

fig, ax = plt.subplots(figsize=(8, 4))
x = np.linspace(0, 2 * np.pi, 100)

for i, color in enumerate(p):
    ax.plot(x, np.sin(x + i * 0.4), color=color.hex,
            linewidth=2.5, label=color.name)

ax.legend(loc="upper right", ncol=2, fontsize=9)
ax.set_title("Using a palette directly")
plt.tight_layout()
plt.show()

## Multiplot

`Multiplot` creates a figure with a GridSpec-based grid. The constructor
accepts `rows`, `columns`, and `width` (in points).  Pre-created axes
are accessed with `m[row, col]`.

In [ ]:
m = Multiplot(rows=2, columns=2, width=700, ratio=0.8)

for r in range(2):
    for c in range(2):
        ax = m[r, c]
        x = np.linspace(0, 2 * np.pi, 100)
        ax.plot(x, np.sin(x + r + c), linewidth=2)
        ax.set_title(f"Subplot ({r},{c})")

plt.tight_layout()
plt.show()

### Custom column / row ratios

In [ ]:
m = Multiplot(rows=1, columns=2, width=800, ratio=0.4,
              width_ratios=[2, 1])

ax1 = m[0, 0]
x = np.linspace(0, 4 * np.pi, 200)
ax1.plot(x, np.sin(x), linewidth=2)
ax1.set_title("Wide plot")

ax2 = m[0, 1]
ax2.bar([1, 2, 3], [4, 7, 2],
        color=["#d32f2f", "#1976d2", "#388e3c"])
ax2.set_title("Narrow plot")

plt.tight_layout()
plt.show()

### Insets

`add_inset(index)` places a small axes inside an existing subplot.

In [ ]:
m = Multiplot(rows=1, columns=1, width=600)

ax = m[0, 0]
x = np.linspace(0, 10, 300)
ax.plot(x, np.sin(x) * np.exp(-x / 5), linewidth=2, color="#1976d2")
ax.set_title("Main plot")

m.add_inset((0, 0), width="30%", height="30%", loc=1)
inset = m[0, 0, 0]
inset.plot(x[:50], np.sin(x[:50]), color="#d32f2f", linewidth=1.5)
inset.set_title("Zoomed in", fontsize=9)

plt.tight_layout()
plt.show()

### Styling axes

`style_axes(index)` configures labels, titles, spines, and ticks in one call.

In [ ]:
m = Multiplot(rows=1, columns=2, width=800, ratio=0.4)

m[0, 0].plot([1, 2, 3], [1, 4, 2], linewidth=2)
m[0, 0].set_title("Default style")

m[0, 1].plot([1, 2, 3], [1, 4, 2], linewidth=2)
m.style_axes(
    (0, 1),
    title="Custom style",
    xlabel="x", ylabel="y",
    labelsize=12,
    hide_spines=["top", "right"],
)

plt.tight_layout()
plt.show()

## chromatify decorator

Wrap any function that **returns an Axes** with `@chromatify` to auto-apply
pychromatic colours, spine styling, and tick formatting.

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))
x = np.linspace(0, 2 * np.pi, 100)
for i in range(6):
    ax.plot(x, np.sin(x + i * 0.5), linewidth=2, label=f"Series {i+1}")
ax.legend()
ax.set_title("Default matplotlib colors")
plt.tight_layout()
plt.show()

In [ ]:
@chromatify
def my_plot():
    fig, ax = plt.subplots(figsize=(8, 4))
    x = np.linspace(0, 2 * np.pi, 100)
    for i in range(6):
        ax.plot(x, np.sin(x + i * 0.5), linewidth=2, label=f"Series {i+1}")
    ax.legend()
    ax.set_title("With @chromatify")
    plt.tight_layout()
    return ax

my_plot()
plt.show()

## Lighter & Darker Palettes

`Palette.lighter()` and `Palette.darker()` return **new** palettes with all
colours brightened or darkened. This is perfect for pairing bold line colours
with pale versions for error bands or shaded regions.

In [ ]:
p = Palette("default")
p_light = p.lighter(0.6)   # pastel versions of every colour

fig, ax = plt.subplots(figsize=(8, 4))
x = np.linspace(0, 4 * np.pi, 200)

for i, (color, light) in enumerate(zip(p, p_light)):
    y = np.sin(x + i * 0.8)
    ax.fill_between(x, y - 0.3, y + 0.3, color=light.hex, alpha=0.5)
    ax.plot(x, y, color=color.hex, linewidth=2, label=color.name)

ax.legend(ncol=2, fontsize=8)
ax.set_title("Bold lines + lighter palette for error bands")
plt.tight_layout()
plt.show()

In [ ]:
# Show original vs lighter vs darker hex values
p = Palette("default")
print("Original:", [c.hex for c in p])
print("Lighter: ", [c.hex for c in p.lighter(0.6)])
print("Darker:  ", [c.hex for c in p.darker(0.3)])

## BrokenAxes

Create axis-break plots for data with discontinuous ranges.

### Broken X-axis

Pass `x` as four values `[lo1, hi1, lo2, hi2]` and `y` as `[lo, hi]`.

In [ ]:
ba = BrokenAxes(x=[0, 5, 15, 20], y=[-1.2, 1.2], width=600)

x1 = np.linspace(0, 5, 50)
x2 = np.linspace(15, 20, 50)

# axes[1] = left panel, axes[2] = right panel
ba.axes[1].plot(x1, np.sin(x1), color="#1976d2", linewidth=2)
ba.axes[2].plot(x2, np.sin(x2), color="#1976d2", linewidth=2)

# axes[0] is the hidden master — use it for overall title
ba.axes[0].set_title("Broken X-Axis", fontsize=14, pad=15)
plt.show()

### Broken Y-axis

Pass `y` as four values `[lo1, hi1, lo2, hi2]` and `x` as `[lo, hi]`.

In [ ]:
x_vals = np.arange(10)
y_vals = np.array([2, 3, 2.5, 3.5, 2, 3, 50, 55, 52, 48])

ba = BrokenAxes(x=[0, 9], y=[0, 5, 45, 60], width=600)

ba.axes[1].plot(x_vals, y_vals, "o-", color="#d32f2f", linewidth=2)
ba.axes[2].plot(x_vals, y_vals, "o-", color="#d32f2f", linewidth=2)

ba.axes[0].set_title("Broken Y-Axis", fontsize=14, pad=15)
plt.show()

### Broken axes inside a Multiplot

Embed broken axes in one cell of a Multiplot grid.

In [ ]:
m = Multiplot(rows=1, columns=2, width=800, ratio=0.5)

# regular plot
ax = m[0, 0]
x = np.linspace(0, 2 * np.pi, 100)
ax.plot(x, np.sin(x), linewidth=2, color="#1976d2")
ax.set_title("Normal")

# broken x-axis
m.add_brokenaxes((0, 1), x=[0, 5, 15, 20], y=[-1.2, 1.2])
bax = m[0, 1, 0]  # left panel
bbx = m[0, 1, 1]  # right panel

x1 = np.linspace(0, 5, 50)
x2 = np.linspace(15, 20, 50)
bax.plot(x1, np.sin(x1), color="#d32f2f", linewidth=2)
bbx.plot(x2, np.sin(x2), color="#d32f2f", linewidth=2)

plt.show()